In [1]:
import clingo
import re
import time

In [2]:
def combine(encoding_path, benchmark_path, destination_path):
    
    combined_path = destination_path + "/combined.lp"
    
    encoding_file = open(encoding_path, "r")
    encoding = encoding_file.read()
    encoding_file.close()
    
    benchmark_file = open(benchmark_path, "r")
    benchmark = benchmark_file.read()
    benchmark_file.close()
    
    combined_file = open(combined_path, "w")
    combined = combined_file.write(benchmark + encoding)
    combined_file.close()

In [3]:
def save_plans(plans, k=0):
    if type(plans)==list:
        for i in range(len(plans)):
            if type(plans[i])==str:
                with open("./python/solution_%d_%d.lp" % (k,i), 'w') as f:
                    f.write(plans[i])
            else:
                if type(plans[i])==list:
                    k+=1
                    save_plans(plans[i],k)
    else:
        if type(plans)==str:
                with open("./python/solution_%d.lp" % (k), 'w') as f:
                    f.write(plans)

In [4]:
def solve(robot_id, destination_path, restrictions=[], fin=[]):
    """
    if fin:
        fin=sum(fin)
    else:
        fin=5
    """
    combined_path = destination_path + "/combined.lp"
    combined_file = open(combined_path, "r")
    combined = combined_file.read()
    combined_file.close()
    
    rid = "#const rid = {}.".format(robot_id)
    #fin = "#const fin = {}.".format(fin)
    fin=""
    for i in range(1,100):
        solution = ""
        horizon = "#const horizon = {}.".format(i)
        asp = horizon + "\n" + rid + "\n" + fin + "\n" + combined
        if restrictions:
            for j in restrictions:
                if j:
                    asp = asp + "\n" + j
            
        ctl = clingo.Control()
        ctl.add("base", [], asp)
        ctl.ground([("base", [])])
        with ctl.solve(yield_=True) as handle:
            for m in handle: solution=("{} ".format(m))
            handle.get()
        solution = solution.replace(' ', '. ')
        if solution:
            return solution, i
            break

In [5]:
def prep(robots, destination, restrictions=[]):
    plans=[]
    plan_=""
    paths = []
    actions = []
    
    if not paths:
        for i in robots:
            paths.append([])

    for i in robots:
        temp = solve(i, destination, restrictions)
        paths[i-1].append(temp[0])
        actions.append(temp[1])
        if paths[i-1][0]:
            plan_ = plan_ + paths[i-1][0]
    return paths, plan_, actions

In [6]:
def collisions(robot_id, encoding_path, plan):
    
    encoding_file = open(encoding_path, "r")
    encoding = encoding_file.read()
    encoding_file.close()
    
    rid = "#const rid = {}.".format(robot_id)
    asp = rid + "\n" + plan + "\n" + encoding
    solution_ = ""
    
    ctl = clingo.Control()
    ctl.add("base", [], asp)
    ctl.ground([("base", [])])
    with ctl.solve(yield_=True) as handle:
        for m in handle: solution_=("{} ".format(m))
        handle.get()
    if solution_== " ":
        return
    else:
        solution_ = solution_.replace(' ', '. ')
        return solution_

In [7]:
def plan(robots, destination, collision, paths, plan_, actions, restrictions=[], actions_=[]):
    if not actions_ or sum(actions_)==0:
        actions_ = list(actions)
    
    max_value = max(actions_)
    max_index = actions_.index(max_value)
    cur_rest = []
    cur_rest.append(collisions(actions_[max_index], collision, plan_))
    actions_[max_index] = 0
    #fin=[min(actions)]
    if cur_rest[0]:
        restrictions.append(cur_rest[-1])
        restrictions_ = " ".join(cur_rest)
        restricted_robots = []
        restricted_robots =  re.findall('s\((.+?),', restrictions_)
        restricted_robots = list(map(int, restricted_robots))
        restricted_robots = list(set(restricted_robots))
        for i in restricted_robots:
            temp = solve(i, destination, restrictions)
            #temp = solve(i, destination, restrictions, fin)
            paths[i-1] = [temp[0]]
            actions[i-1] = temp[1]
        plan_ = ""
        for i in robots:
            if paths[i-1][0]:
                plan_ = plan_ + paths[i-1][0]
        plan_ = plan(robots, destination, collision, paths, plan_, actions, restrictions, actions_)
    return plan_

In [8]:
enc = "./asprilo-encodings/shortest_path/pathfinder.lp"
ben = "./instances/example4/instance.lp"
dest = "./python"
col = "./asprilo-encodings/collision/collision.lp"
robots = []
for i in range(1,26):
    robots.append(i)
start = time.time()
rest=[]
sel=[]
plans =[]


combine(enc, ben, dest)
paths, plan_, actions = prep(robots, dest, rest)
plans_=plan(robots, dest, col, paths, plan_, actions, rest, sel)
save_plans(plans_)
end = time.time()
print(end-start)

31.557743310928345
